# Fast-Neutron Lab
## Data analysis code and lab notebook

### Introduction
* This 'Jupyter Notebook' is your manual, data analysis and log book all at the same time. It consists of individual cells of text or Python code and you can edit or add cells to your heart's content.
* When using the notebook, there are two modes: *command* and *edit* mode. In command mode, you can add, move, run or delete cells. In edit mode, you modify its content. Use the *Esc* key or click outside the cell to leave edit mode. With *shift* + *return* you can run it directly from edit mode.
* When you 'run' a cell it will either be formated or executed depending on whether it is a text cell or a code cell, respectivly. The output will be shown inside the notebook.
* It is best to run the notebook piece-by-piece as we are progressing through the lab. If something doesn't work as expected, try 'Kernel->Restart' to get a fresh working environment.
* Code cells support typical editing features of an IDE, such as tab-complete. You find more info in the 'Help' menu including all keyboard shortcuts!
* **Please note**: The code is not supposed to work out-of-the-box but needs some tweaks from your side to complete all the way through! So if you encounter an error, don't despair, try to fix it and ask for help should you get stuck!

## How to transfer data for the analysis
The data is stored in .root files to be used with the *ROOT* data analysis framework. 
To analyse the data here instead, we need to extract the histogrammed data from the file into a format we can easliy import.

For this, use the command *ROOTtoCSV*. 
The prgrams expect two input parameters the name of the *ROOT* file and the name of the histogram you want to extract.

Then copy the data into the folder where you run this Jupyter Notebook from -- or use the upload function in the main Jupyter screen.

## How to run this analysis
Some initialization in your program:
To plot things, you use the matplotlib library. You need to import the library before the first use and initialize the interactive plots inside the notebook:


In [ ]:
import numpy as np
import scipy
# enable interactive notebook plots:
# (alternative: use 'inline' instead of 'notebook' for static images)
%matplotlib notebook
import matplotlib.pyplot as plt

# Plotting the time-of-flight data

* The aim is to transform the TDC spectrum into a TOF spectrum.
* First you need to import the data and create a histogram.
* There is already a class ready to be used called histhelpers.

In [ ]:
import histhelpers
from histhelpers import getBinCenters
# enter the data from TDC YAP 0
tof = histhelpers.import_hist_from_csv('data/filename.csv') # TODO: put correct file name!
print("Loaded file '{}' imported from '{}'".format(tof.filename, tof.name))


fig, axes = plt.subplots() # makes new plot
plt.yscale('log')
axes.plot(getBinCenters(tof.bins), tof.data, label="NE213 with PuBe", drawstyle="steps")
plt.xlabel('Channel')
plt.ylabel('Counts')

plt.legend()

# Converting the TDC spectrum a to time-of-flight spectrum

* To determine the $T_0$ reference point you need to find the gamma-flash.
* The precise peak position of the gamma flash can be found by fitting a Gaussian to the data.
* First plot the data:

In [ ]:
fig, axes = plt.subplots()

# plot data
plt.yscale('log') # comment out for lin scale
plt.ylim(ymin=1, ymax=max(tof.data))
axes.plot(getBinCenters(tof.bins), tof.data, label="TOF data", ds="steps") # ls == linestyle

Now adjust the arguments of the routine *fit_gaussians_at_pos* from the *fithelpers.py* file to match the gamma flash position:

In [ ]:
import fithelpers
# TODO: adjust position to be fitted in function's argument below
gammaflash_fit = fithelpers.fit_gaussian_at_pos(getBinCenters(tof.bins), tof.data, pos=100)
print(f"gammaflash_fit1: {gammaflash_fit.mu}")
axes.plot(getBinCenters(tof.bins), gammaflash_fit.value(getBinCenters(tof.bins)), label="Gauss Fit")

## Optional Challenge: perform the fit on your own!
In the previous cell we "cheated" a bit using ready-made helper code; why not set the parameters yourself and run [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) yourself? You'll find the necessary inspiration in the *Introduction for FYSC12 Labs* Jupyter Notebook you've received.

*Discuss/state below*: What are the uncertainties of the fitted parameters? Is this better than what you would expect reading the values off by eye?

In [ ]:
from scipy.optimize import curve_fit #importing fitting library
x_val = tof.bins #TOF values
y_val = tof.data #number of counts

#....... YOUR CODE HERE .......


## Calcuating $T_0$

Now put the correct values in to calcuate the reference time $T_0$:

**Optional**: Document how you determine `seconds_per_TDC_channel` by e.g. creating a new cell where you load the TDC calibration data.

In [ ]:
#Enter the distance from detector to neutron source:
distance =  100 # in [m]; TODO: measure and put right value here!
speed_of_light = 2.99792458e8 # in [m/s]
gammaflash_position = gammaflash_fit.mu # replace with value read off by eye in case you skipped the fit
seconds_per_TDC_channel = 1e-9 # in s/ch; TODO: measure and put right value here!


T0 = # TODO: add a calculation here!
print ("T0 calculated to be at: {} [ch]".format(T0))

# DRAW A LINE AT T0:
axes.text(T0, max(tof.data)*.7, "T0")
axes.plot((T0, T0), (1, max(tof.data)*.7), 'k-')

For clarity, let's put the axis into the correct time binning, invert it and shift the zero position to T0

In [ ]:
# copy spectrum into new data structure (before messing with it)
import copy
tof_in_ns = copy.deepcopy(tof)

# now align the axis so that 0 =^= T0 and convert TDC channel to time [ns]
tof_in_ns.bins = ((-1) * tof.bins + T0) * seconds_per_TDC_channel
tof_in_ns.bins *= 1e9 # convert s to ns

# plot the result
fig, axes = plt.subplots()
axes.plot(getBinCenters(tof_in_ns.bins), tof_in_ns.data, label="summed TOF data", ds="steps")
plt.xlabel('time of flight [ns]')
plt.ylabel('counts')
plt.yscale('log')
plt.legend()

# Clean-up of the spectra and background subtraction
Let's remove "unphysical" part with $t<0$ of the ToF spectrum

In [ ]:
# make copy of spectrum first
tof_stripped = copy.deepcopy(tof_in_ns)

## remove some of the data that would mess up our calculation:
## - find all bins with x > 0, remember last index (just for the very curious -- why last index and not first: the order in memory stays but we flipped the axis when we multiplied with -1 above)
idx_low = np.where(tof_stripped.bins>0)[0][-1]
#idx_high = np.where(Tn.bins>100)[0][-1] # could be used to remove upper part (replaces '0' below)

## strip all other bins to remove those with x < 0
tof_stripped.bins = tof_stripped.bins[0:idx_low]
tof_stripped.data = tof_stripped.data[0:idx_low-1] # mind the offset, len(bins) = len(data)+1 !
# plot the result
fig, axes = plt.subplots()
axes.plot(getBinCenters(tof_stripped.bins), tof_stripped.data, label="summed TOF data", ds="steps")
plt.xlabel('time of flight [ns]')
plt.ylabel('counts')
#plt.yscale('log')
plt.legend()

## Optional Task: Subtract the random background from the spectrum (you still need to run the cell, even if you don't touch this part otherwise)

Think about (and answer below): How can you best determine the amount of background? What are the implications of removing it? Is this a correction or a purely cosmetic step?

In [ ]:
tof_wo_bckgrd = copy.deepcopy(tof_stripped)
# subtract background of random triggers
tof_wo_bckgrd.data -= 0 # TODO: determine (fit?) random background and put here

# values < 0 are unphysical -- set to zero instead
# tof_wo_bckgrd.data[tof_wo_bckgrd.data < 0] = 0

# plot the result
fig, axes = plt.subplots()
plt.ylim(0.1,max(tof_wo_bckgrd.data))
axes.plot(getBinCenters(tof_wo_bckgrd.bins), tof_wo_bckgrd.data, ds="steps", label="TOF data w/o background")
plt.xlabel('time of flight [ns]')
plt.ylabel('counts')
plt.yscale('log')
plt.legend()

# Convert the time-of-flight to an energy spectrum

In [ ]:
## constants:
#Add neutron mass
M_n = 1e6  ## mass of n in MeV/c**2; TODO: put right value here!

Tn = copy.deepcopy(tof_wo_bckgrd)
Tn.bins *= 1e-9 # convert to seconds

# show the raw data once more for clarity
fig, axes = plt.subplots()
axes.plot(getBinCenters(Tn.bins), Tn.data, label="data (cut)", drawstyle="steps")
plt.xlabel('time of flight [s]')
plt.ylabel('counts')

## non-relativistic/classic approach:
Tn_cls = copy.deepcopy(Tn)
Tn_cls.bins = 0.5 * (M_n / pow (speed_of_light, 2)) * ( pow(distance, 2) / pow(Tn_cls.bins, 2) )
## relativistic approach: B. Nilsson PhD thesis, LU, 2003, equation 2.7
Tn_rlv = copy.deepcopy(Tn)
beta = 1. / ( 1. + ((Tn_rlv.bins - distance / speed_of_light) * speed_of_light / distance))
Tn_rlv.bins = M_n * ( ( 1. / np.sqrt(1. - pow(beta, 2))) - 1. )

# plot the result
fig, axes = plt.subplots()
axes.plot(getBinCenters(Tn_cls.bins), Tn_cls.data, label="data (classic)", drawstyle="steps")
axes.plot(getBinCenters(Tn_rlv.bins), Tn_rlv.data, label="data (relativistic)", drawstyle="steps")

plt.xlabel(r'$T_n \mathrm{[MeV]}$')
plt.ylabel('counts')
plt.yscale('log')
plt.ylim(ymin=0.1)
plt.title("Neutron kinectic energy")
plt.legend()

# Discussion

Please answer the questions below, make sure that all plots above look good (i.e. show what you want to highlight and are labeled) and submit your Jupyter Notebook.

1. what parameters enter into the calculation to get the energy spectrum from the original data recorded by the TDC?
2. what are their uncertainties and how do they affect the result? How can you practically estimate this effect?
* what would you say affects the efficiency of detecting a neutron with today's setup?
* how can you convince your colleagues that you have measured neutrons today?
* what is the background in this technique and how does it affect your measurement?
* compared to charged particles, why is it so difficult to measure the neutron's energy in general?
* what is the function of the CFD ("constant fraction descriminator") module in the setup? Do we actually need it?
* can you come up with suggestions for improvment of this technique?

... and: would you be interested in learning more in an *Advanced Neutron Tagging Lab*? ;)

.... YOUR ANSWERS HERE ....